# Import Libraries

In [103]:
import os
import pandas as pd
import numpy as np

# Load dataset and filter

In [29]:
cwd = os.getcwd()

simp_path4 = 'raw_data/df.csv'
abs_path4 = os.path.abspath(os.path.join(cwd,'..',simp_path4))
df_500 = pd.read_csv(abs_path4)
df_500.date = pd.to_datetime(df_500.date)

In [30]:
df_500['year'] = pd.DatetimeIndex(df_500['date']).year
df_500['month'] = pd.DatetimeIndex(df_500['date']).month
df_500['day'] = pd.DatetimeIndex(df_500['date']).day

In [31]:
df_500 = df_500[df_500.year>1999]

In [32]:
list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']

In [33]:
df_top10 = df_500[df_500['ticker'].isin(list_of_10_stocks)]

In [34]:
df_top10

,ticker,open,close,adj_close,low,high,volume,date,sector,year,month,day
4815,AAPL,3.745536,3.997768,2.686497,3.631696,4.017857,133949200,2000-01-03,TECHNOLOGY,2000,1,3
4816,AAPL,3.866071,3.660714,2.459998,3.613839,3.950893,128094400,2000-01-04,TECHNOLOGY,2000,1,4
4817,AAPL,3.705357,3.714286,2.495997,3.678571,3.948661,194580400,2000-01-05,TECHNOLOGY,2000,1,5
4818,AAPL,3.790179,3.392857,2.279998,3.392857,3.821429,191993200,2000-01-06,TECHNOLOGY,2000,1,6
4819,AAPL,3.446429,3.553571,2.387998,3.410714,3.607143,115183600,2000-01-07,TECHNOLOGY,2000,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...
3275819,UNH,263.000000,262.399994,262.399994,261.570007,263.489990,1786200,2018-08-20,HEALTH CARE,2018,8,20
3275820,UNH,261.850006,261.690002,261.690002,260.730011,262.519989,1559300,2018-08-21,HEALTH CARE,2018,8,21
3275821,UNH,261.950012,260.739990,260.739990,260.250000,261.950012,2148500,2018-08-22,HEALTH CARE,2018,8,22
3275822,UNH,261.019989,261.420013,261.420013,259.559998,262.559998,1603500,2018-08-23,HEALTH CARE,2018,8,23


In [36]:
df_T = df_top10[df_top10['ticker'] == "T"]

In [42]:
len(df_T)

4692

# Create Sequences for LSTM

-filter only 14 years

In [40]:
df_top10_14y = df_top10[df_top10.year<2015]

In [41]:
df_top10_14y['ticker'].unique()

array(['AAPL', 'AMZN', 'JPM', 'INTC', 'ADBE', 'MA', 'T', 'NVDA', 'PG',
       'UNH'], dtype=object)

In [43]:
df_T = df_top10_14y[df_top10_14y['ticker'] == "T"]

In [44]:
len(df_T)

3773

-each day

In [47]:
df_T

,ticker,open,close,adj_close,low,high,volume,date,sector,year,month,day
2431677,T,48.562500,47.000000,19.225044,46.625000,49.000000,5791900,2000-01-03,PUBLIC UTILITIES,2000,1,3
2431678,T,47.062500,44.250000,18.100174,43.500000,47.750000,7173600,2000-01-04,PUBLIC UTILITIES,2000,1,4
2431679,T,45.000000,44.937500,18.381397,44.437500,45.250000,9090000,2000-01-05,PUBLIC UTILITIES,2000,1,5
2431680,T,44.562500,43.750000,17.993273,43.125000,45.000000,7153600,2000-01-06,PUBLIC UTILITIES,2000,1,6
2431681,T,44.562500,44.125000,18.147497,43.375000,44.750000,5885600,2000-01-07,PUBLIC UTILITIES,2000,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...
2435445,T,34.020000,33.959999,27.890707,33.930000,34.189999,8831900,2014-12-24,PUBLIC UTILITIES,2014,12,24
2435446,T,34.049999,34.169998,28.063179,33.980000,34.230000,11486800,2014-12-26,PUBLIC UTILITIES,2014,12,26
2435447,T,34.070000,34.110001,28.013905,34.020000,34.270000,12708100,2014-12-29,PUBLIC UTILITIES,2014,12,29
2435448,T,34.090000,34.080002,27.989269,34.029999,34.200001,12369700,2014-12-30,PUBLIC UTILITIES,2014,12,30


In [136]:
stock = 'T'
sequence_a = []
y = []
for i in range(0, 3600, 36):
    day = []
    df_ = df_top10_14y[df_top10_14y['ticker'] == stock]
    df_ = df_.iloc[i:i+37,3:6]
    for p in range(36):
        day.append(df_.iloc[p,:])       
    sequence_a.append(day)
    y.append(df_.iloc[36,0])

In [137]:
len(y)

100

In [138]:
X = np.array(sequence_a).astype(np.float32)

In [139]:
X.shape

(100, 36, 3)

In [140]:
X[2]

array([[17.6272  , 42.      , 46.625   ],
       [17.31704 , 40.875   , 43.5625  ],
       [17.161955, 40.4375  , 41.9375  ],
       [17.34288 , 41.125   , 42.625   ],
       [17.032724, 40.9375  , 42.375   ],
       [17.472113, 41.      , 42.4375  ],
       [18.5835  , 42.4375  , 45.4375  ],
       [18.195808, 43.5     , 45.1875  ],
       [17.885656, 43.      , 44.25    ],
       [18.118275, 43.25    , 44.3125  ],
       [18.454275, 43.6875  , 45.125   ],
       [17.523804, 41.75    , 44.5     ],
       [17.34288 , 41.6875  , 42.6875  ],
       [16.981031, 40.8125  , 42.      ],
       [17.782269, 41.0625  , 43.1875  ],
       [17.730572, 41.875   , 42.9375  ],
       [17.497963, 41.875   , 43.5     ],
       [18.661043, 42.25    , 46.      ],
       [18.841972, 45.1875  , 45.9375  ],
       [18.544718, 44.1875  , 45.      ],
       [19.022894, 44.      , 46.75    ],
       [18.893665, 45.3125  , 46.9375  ],
       [18.738588, 44.8125  , 45.4375  ],
       [18.299187, 43.9375  , 45.5

In [141]:
y[2]

19.1779766082764

In [143]:
df_T.iloc[107:110,:]

,ticker,open,close,adj_close,low,high,volume,date,sector,year,month,day
2431784,T,44.6875,45.3125,18.738588,44.6250,46.3750,5289900,2000-06-06,PUBLIC UTILITIES,2000,6,6
2431785,T,45.2500,46.3750,19.177977,45.0625,46.7500,4645500,2000-06-07,PUBLIC UTILITIES,2000,6,7
2431786,T,46.3750,47.0000,19.436434,46.3125,47.3125,4517900,2000-06-08,PUBLIC UTILITIES,2000,6,8


In [ ]:
import numpy as np

# --- SEQUENCE A (AMZN)

day_1 = [10, 25, 50]  # OBSERVATION 1 [adj_close, low, high]
day_2 = [13, 10, 70]  # OBSERVATION 2 [adj_close, low, high]
day_3 = [ 9,  5, 90]
day_4 = [ 7,  0, 95]

sequence_a = [day_1, day_2, day_3, day_4]

y_a = 110 # adj_close of +1, +7, +21, +63

# --- SEQUENCE B (T)
sequence_b = [[25, 20, 30], [26, 24, 50], [28, 20, 80], [22, 3, 110]]
y_b = 125

# --- SEQUENCE C (AAPL)
sequence_c = [[15, 10, 60], [25, 20, 65], [35, 10, 75], [36, 15, 70]]
y_c = 30

X = np.array([sequence_a, sequence_b, sequence_c]).astype(np.float32)
y = np.array([y_a, y_b, y_c]).astype(np.float32)

print(X.shape)
print(y.shape)

In [7]:
from stock_prices_predictions.get_data import get_lstm_training_data

X, y = get_lstm_training_data('AAPL', 5)

In [8]:
print(y)

[3.12749695777893, 2.85299777984619, 2.2679979801178, 2.30399775505066, 2.56799745559692, 0.971999287605286, 0.794999241828918, 0.899999260902405, 1.16399872303009, 0.982558786869049, 0.935999095439911, 0.745919287204742, 0.937439382076263, 1.01519906520844, 1.17023897171021, 1.1500791311264, 0.821759343147278, 0.719999194145203, 0.696479320526123, 0.745439231395721, 0.70127946138382, 0.69743937253952, 0.674879372119904, 0.918719112873077, 0.943678915500641, 0.997919201850891, 0.978239238262177, 1.16159880161285, 1.24799871444702, 1.26959884166718, 1.55183851718903, 1.48847877979279, 1.85615825653076, 2.94479751586914, 3.35039734840393, 3.89087653160095, 3.41183638572693, 3.61055588722229, 4.16447639465332, 5.14655447006226, 6.23615455627441, 8.21663188934326, 6.30335378646851, 6.75743341445923, 5.51711511611938, 6.15071439743042, 7.18655252456665, 8.50559043884277, 9.11519050598145, 8.4451150894165, 9.54911231994629, 11.8943872451782, 12.2678279876709, 15.160306930542, 16.467828750610

In [9]:
X[0]

array([[2.6864974, 3.6316965, 4.017857 ],
       [2.459998 , 3.6138394, 3.950893 ],
       [2.4959974, 3.6785715, 3.9486606],
       [2.279998 , 3.392857 , 3.8214285],
       [2.3879979, 3.4107144, 3.607143 ],
       [2.3459978, 3.3839285, 3.6517856],
       [2.2259984, 3.232143 , 3.549107 ],
       [2.092498 , 3.0892856, 3.4107144],
       [2.3219984, 3.3035715, 3.5267856],
       [2.4104974, 3.549107 , 3.6517856],
       [2.4944973, 3.5870535, 3.7857144],
       [2.5574977, 3.6919644, 3.8839285],
       [2.7239976, 4.053571 , 4.339286 ],
       [2.671498 , 3.935268 , 4.080357 ],
       [2.5499978, 3.7544644, 4.026786 ],
       [2.6939976, 3.65625  , 4.040179 ],
       [2.6444976, 3.919643 , 4.078125 ],
       [2.6399975, 3.8214285, 4.035714 ],
       [2.438998 , 3.59375  , 3.9598215],
       [2.4899974, 3.375    , 3.7098215],
       [2.4059975, 3.5714285, 3.75     ],
       [2.3714983, 3.4642856, 3.6473215],
       [2.4794974, 3.580357 , 3.7232144],
       [2.5919971, 3.700893 , 3.92